# Algorithmically Assign Abx To CSNs based on Model Predictions

In [2]:
import numpy as np
import pandas as pd
from pulp import *

### Load Predictions Data For Each Abx Option

In [ ]:
base_path="/home/conorcorbin/repos/er_infection/results/testing/{abx}"
abx_options = []
df = pd.DataFrame()
for i, abx in enumerate(abx_options):
    path = base_path.format(abx=abx)
    if i == 0:
        df = pd.read_csv(path)
        df = df[['anon_id', 'pat_enc_csn_id_coded', 'label', 'predictions']]
        df = df.rename(columns={'label' : '%s_label' % abx, '%s_predictions' % abx})
    else:
        df_preds = pd.read_csv(path)
        df_preds = df_preds[['anon_id', 'pat_enc_csn_id_coded', 'label', 'predictions']]
        df_preds = df_preds.rename(columns={'label' : '%s_label' % abx, '%s_predictions' % abx})
        df = df.merge(df_preds, how='left', on='pat_enc_csn_id_coded')


### Create Binary Integer Programming Problem Formulation

In [ ]:
abx_model = pulp.LpProblem("Antibiotics Selection Problem", pulp.LpMaximize)
abx_options = []

# Create binary indicators for whether treatment is used
drug_inds = {}
for abx in abx_options:
    drug_inds[abx] = [LpVariable('%s_%d' % (abx, i), lowBound=0, upBound=1, cat='Binary')
                      for i in range(len(df))]

# Add objective function to model
per_csn_sum = []
for i in range((len(df))):
    _sum = 0
    for abx in abx_options:
        _sum += drug_inds[abx][i] * df['%s_predictions' % abx].values[i]
    per_csn_sum.append(_sum)
    
abx_model += lpSum(per_csn_sum)

# Add one selection constraint
for i in range(len(df)):
    selections = []
    for abx in abx_options:
        selections.append(drug_inds[abx][i])
    abx_model += lpSum(selections) == 1

# Add max assignment constraints
abx_assignment_constraints = {"Vancomycin" : 100,
                              "Ampicillin" : 100,
                              "Cefazolin" : 100,
                              "Ceftriaxone" : 100,
                              "Cefepime" : 100,
                              "Zosyn" : 100,
                              "Ciprofloxacin" : 100,
                              "Meropenem" : 100,
                              "Vancomycin_Meropenem" : 100,
                              "Vancomycin_Zosyn" : 100,
                              "Vancomycin_Cefepime" : 100,
                              "Vancomycin_Ceftriaxone" : 100
                             }

for drug in drug_inds:
    abx_model += lpSum([drug_inds[drug][i] for i in range(len(df))]) == abx_assignment_constraints[drug]

# Solve model
antibiotics_model.solve()
print("Status:", LpStatus[antibiotics_model.status])


In [ ]:
# Scratch work don't throw out yet
# Create list of contiuous variables
pip_ind = [LpVariable('pip_%d' % i, lowBound=0, upBound=1, cat='Binary') for i in range(len(data_small))]
cefepime_ind = [LpVariable('cefe_%d' % i, lowBound=0, upBound=1, cat='Binary') for i in range(len(data_small))]
ceftriaxone_ind = [LpVariable('ceft_%d' % i, lowBound=0, upBound=1, cat='Binary') for i in range(len(data_small))]

antibiotics_model += lpSum([pip_ind[i]*data_small['p_of_c_piptazo'].values[i] + \
                        cefepime_ind[i]*data_small['p_of_c_cefepime'].values[i] + \
                        ceftriaxone_ind[i]*data_small['p_of_c_ceftriaxone'].values[i] 
                        for i in range(len(data_small))]), "Total cumulative probability"

# Add row constraints - ie rows must sum to one
for i in range(len(data_small)):
    antibiotics_model += lpSum([pip_ind[i], cefepime_ind[i], ceftriaxone_ind[i]]) == 1

# Add Column constraints for each antibiotic'
antibiotics_model += lpSum([pip_ind[i] for i in range(len(data_small))]) == 174
antibiotics_model += lpSum([cefepime_ind[i] for i in range(len(data_small))]) == 35
antibiotics_model += lpSum([ceftriaxone_ind[i] for i in range(len(data_small))]) == 561
